## 1) Imports & Constants

Load core libraries and set configuration:

- `START`, `END`: season range (2000–2024)  
- `RAW_REG_DIR`, `RAW_PO_DIR`: where to save raw CSVs  
- `DB_FILE`: path to the DuckDB database file  
- `SLEEP`: seconds to pause between requests to avoid rate-limiting


In [1]:
import pathlib, time, duckdb, pandas as pd, requests
from random import randint


START, END  = 2000, 2024
RAW_REG_DIR = pathlib.Path("data/raw/regular")
RAW_PO_DIR  = pathlib.Path("data/raw/playoffs")
DB_FILE     = pathlib.Path("../data/processed/nba.duckdb")


SLEEP = 1.0       


In [2]:
import os
print(os.getcwd())


/Users/apple/Documents/nba-playoff-value/notebooks


## 2) Helper Functions

- `html_url()`: construct the correct Basketball-Reference URL for a given year and mode  
- `fetch_advanced()`: download the “Advanced” table, clean duplicate headers, and tag with season/playoffs


In [3]:
def html_url(year: int, playoffs: bool):
    return (
        f"https://www.basketball-reference.com/playoffs/NBA_{year}_advanced.html"
        if playoffs
        else f"https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html"
    )

def fetch_advanced(year: int, playoffs: bool):
    url = html_url(year, playoffs)
    print(url)
    try:
         df = pd.read_html(url, match="Advanced")[0]
    except ValueError:
        print(f"   advanced table missing for {year} ({'PO' if playoffs else 'RS'})")
        return None

    # drop the extra header rows that repeat "Rk" (Rk stands for rank)
    df = df[df["Rk"] != "Rk"].copy()
    df["Season"]   = year
    df["Playoffs"] = playoffs
    time.sleep(SLEEP)
    return df


## 3) Download Loop
- Loop through each season and mode to fetch and save the advanced-stats table as a .csv file
- Skip missing tables and pause `SLEEP` seconds between requests
 

In [4]:
for yr in range(START, END + 1):
    for po in (False, True):
        df = fetch_advanced(yr, po)
        if df is None:       # skip missing seasons
            continue
        target_dir = RAW_PO_DIR if po else RAW_REG_DIR
        target_dir.mkdir(parents=True, exist_ok=True)
        df.to_csv(target_dir / f"{yr}.csv", index=False)
        print(f"    saved {yr} {'PO' if po else 'RS'} ({len(df)} rows)")
        time.sleep(SLEEP)
print("Success: All CSVs saved")



https://www.basketball-reference.com/leagues/NBA_2000_advanced.html
    saved 2000 RS (497 rows)
https://www.basketball-reference.com/playoffs/NBA_2000_advanced.html
    saved 2000 PO (181 rows)
https://www.basketball-reference.com/leagues/NBA_2001_advanced.html
    saved 2001 RS (538 rows)
https://www.basketball-reference.com/playoffs/NBA_2001_advanced.html
    saved 2001 PO (190 rows)
https://www.basketball-reference.com/leagues/NBA_2002_advanced.html
    saved 2002 RS (501 rows)
https://www.basketball-reference.com/playoffs/NBA_2002_advanced.html
    saved 2002 PO (186 rows)
https://www.basketball-reference.com/leagues/NBA_2003_advanced.html
    saved 2003 RS (484 rows)
https://www.basketball-reference.com/playoffs/NBA_2003_advanced.html
    saved 2003 PO (187 rows)
https://www.basketball-reference.com/leagues/NBA_2004_advanced.html
    saved 2004 RS (586 rows)
https://www.basketball-reference.com/playoffs/NBA_2004_advanced.html
    saved 2004 PO (189 rows)
https://www.basketball-re

## 4) Create SQL tables with DuckDB
- Erase previous tables for regular and playoffs if they exist from previous runs for clean loads
- Create regular and playoff tables from bulk load .csv files in respective folders
- Count rows for sanity check before analysis

In [5]:
con = duckdb.connect(DB_FILE)

con.sql("DROP TABLE IF EXISTS adv_regular;")
con.sql("DROP TABLE IF EXISTS adv_playoffs;")

con.sql("""CREATE TABLE adv_regular AS
           SELECT * FROM read_csv_auto('data/raw/regular/*.csv');""")

con.sql("""CREATE TABLE adv_playoffs AS
           SELECT * FROM read_csv_auto('data/raw/playoffs/*.csv');""")

reg_rows = con.sql("SELECT COUNT(*) AS n FROM adv_regular").fetchone()[0]
po_rows  = con.sql("SELECT COUNT(*) AS n FROM adv_playoffs").fetchone()[0]
print(f"Rows loaded -> {reg_rows} regular | {po_rows} playoff")

con.close()


Rows loaded -> 15183 regular | 5082 playoff


## 5) Create Career Difference View

- Aggregate regular-season Win Shares (`reg_ws`) and minutes played (`reg_mp`) by player  
- Aggregate playoff Win Shares (`po_ws`) and minutes played (`po_mp`) by player  
- Join those aggregates into a single view `v_player_career`  
- Compute per-48 metrics (`reg_ws48`, `po_ws48`) and their difference (`ws48_diff`)  
- Print a confirmation once the view is created  


In [6]:
con = duckdb.connect(DB_FILE)
con.sql("""
CREATE OR REPLACE VIEW v_player_career AS
WITH reg AS (
  SELECT Player,
         SUM(WS)       AS reg_ws,
         SUM(MP)       AS reg_mp
  FROM adv_regular
  GROUP BY 1
),
po AS (
  SELECT Player,
         SUM(WS) AS po_ws,
         SUM(MP) AS po_mp
  FROM adv_playoffs
  GROUP BY 1
)
SELECT
  r.Player,
  reg_mp,
  po_mp,
  (po_ws / NULLIF(po_mp,0))*48  AS po_ws48,
  (reg_ws/ NULLIF(reg_mp,0))*48 AS reg_ws48,
  (po_ws / NULLIF(po_mp,0))*48 - (reg_ws/NULLIF(reg_mp,0))*48 AS ws48_diff
FROM reg r JOIN po USING(Player);
""")
print("View created -> v_player_career")
con.close()



View created -> v_player_career


## 6) Preview Top & Bottom Performers

- Query **top 15** players by WS/48 Δ (highest playoff vs. regular improvement)  
- Query **bottom 15** players by WS/48 Δ (largest decline)  
- Round differences to 3 decimals and load into Pandas DataFrames  
- Display both tables for quick inspection  
  


In [7]:
con = duckdb.connect(DB_FILE)

top15 = con.sql("""SELECT Player, ROUND(ws48_diff,3) AS diff
                   FROM v_player_career
                   ORDER BY diff DESC
                   LIMIT 15""").df()
bottom15 = con.sql("""SELECT Player, ROUND(ws48_diff,3) AS diff
                      FROM v_player_career
                      ORDER BY diff ASC
                      LIMIT 15""").df()

display(top15)
display(bottom15)

con.close()


,Player,diff
0,Jordan McRae,2.348
1,Vassilis Spanoulis,1.618
2,Malcolm Delaney,1.577
3,Anthony Roberson,1.161
4,Kevin Knox,1.096
5,Sundiata Gaines,0.647
6,Blake Ahearn,0.621
7,Dalano Banton,0.572
8,Terrel Harris,0.569
9,Elliot Perry,0.513


,Player,diff
0,Harry Giles,-1.275
1,John Wallace,-1.251
2,Tracy Murray,-1.031
3,Cassius Winston,-0.995
4,Stephen Graham,-0.986
5,David Duke Jr.,-0.983
6,Phil Pressey,-0.975
7,D.J. Strawberry,-0.924
8,Wayne Simien,-0.761
9,Kyle O'Quinn,-0.728


## 7) Filter Preview for Top and Bottom Players
- Filter out players with fewer than **1,000 regular-season minutes** and **500 playoff minutes**
  to avoid small sample size variance

In [8]:
con = duckdb.connect(DB_FILE)

filtered_top15 = con.sql("""
  SELECT Player, ROUND(ws48_diff,3) AS diff, reg_mp, po_mp
  FROM v_player_career
  WHERE reg_mp >= 1000
    AND po_mp  >=  500
  ORDER BY diff DESC
  LIMIT 15
""").df()

filtered_bottom15 = con.sql("""
  SELECT Player, ROUND(ws48_diff,3) AS diff, reg_mp, po_mp
  FROM v_player_career
  WHERE reg_mp >= 1000
    AND po_mp  >=  500
  ORDER BY diff ASC
  LIMIT 15
""").df()

display(filtered_top15)
display(filtered_bottom15)

con.close()


,Player,diff,reg_mp,po_mp
0,Andrew Nembhard,0.098,3770.0,554.0
1,Anthony Edwards,0.085,10392.0,1075.0
2,Jason Maxiell,0.071,11379.0,581.0
3,Baron Davis,0.070,31822.0,1851.0
4,Tim Thomas,0.066,22747.0,1485.0
5,Daniel Gibson,0.065,9316.0,881.0
6,Jordan Poole,0.060,9349.0,890.0
7,Jaden McDaniels,0.058,7835.0,667.0
8,Scott Williams,0.056,5575.0,565.0
9,Jameer Nelson,0.055,26959.0,1429.0


,Player,diff,reg_mp,po_mp
0,Stephon Marbury,-0.140,27814.0,612.0
1,Julius Randle,-0.129,20512.0,510.0
2,Terry Porter,-0.117,4585.0,546.0
3,Anthony Peeler,-0.105,10424.0,739.0
4,Tyler Hansbrough,-0.105,7233.0,652.0
5,Evan Fournier,-0.100,21353.0,566.0
6,Jacque Vaughn,-0.094,12152.0,734.0
7,Georges Niang,-0.087,7797.0,774.0
8,Karl-Anthony Towns,-0.085,19455.0,1094.0
9,Joe Harris,-0.083,12298.0,715.0
